In [57]:
import pandas as pd

# 앙상블 학습

In [32]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

In [58]:
df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
df.head()  # 스케일링 필요함

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [33]:
from sklearn.preprocessing import MinMaxScaler  # 스케일링 
scaler = MinMaxScaler()
cancer_scaled = scaler.fit_transform(cancer.data)

In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    cancer_scaled, cancer.target, stratify= cancer.target,  test_size= 0.2, random_state= 2011

)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

## 앙상블 학습을 위한 분류기

#### 로지스틱 회귀
#### 서포트 벡터 머신
#### k 최근접 이웃

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [48]:
lr = LogisticRegression()
svc = SVC()
knn = KNeighborsClassifier()

In [49]:
from sklearn.ensemble import VotingClassifier

vo_clf = VotingClassifier(
    estimators = [('LR',lr), ('SVC',svc), ('KNN',knn)], 
    voting = 'hard'
)

In [50]:
vo_clf.fit(X_train, y_train)
pred = vo_clf.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.956140350877193

## 개별 모델의 학습 / 예측 / 평가

In [52]:
for classifier in [lr, svc, knn]:
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test)
    acc = accuracy_score(y_test, pred)
    classifier_name = classifier.__class__.__name__
    print(f'{classifier_name} 정확도{acc : .4f}')

LogisticRegression 정확도 0.9386
SVC 정확도 0.9649
KNeighborsClassifier 정확도 0.9561


## 소프트 보팅
 ### - 로지스틱 회귀
 ### - k 최근접 이웃

In [54]:
vo_clf = VotingClassifier(
    estimators = [('LR',lr), ('KNN',knn)], 
    voting = 'soft'
)
vo_clf.fit(X_train,y_train)
pred = vo_clf.predict(X_test)
accuracy_score(y_test, pred)

0.956140350877193

In [55]:
vo_clf.predict_proba(X_test)

array([[9.93299003e-01, 6.70099726e-03],
       [9.97860114e-01, 2.13988594e-03],
       [1.55035222e-02, 9.84496478e-01],
       [2.11048802e-02, 9.78895120e-01],
       [9.74826742e-01, 2.51732577e-02],
       [1.18101174e-01, 8.81898826e-01],
       [1.83346646e-02, 9.81665335e-01],
       [1.72260326e-03, 9.98277397e-01],
       [4.17790994e-02, 9.58220901e-01],
       [4.57872746e-02, 9.54212725e-01],
       [2.50799952e-02, 9.74920005e-01],
       [2.98787609e-02, 9.70121239e-01],
       [2.25234183e-03, 9.97747658e-01],
       [7.50971304e-01, 2.49028696e-01],
       [9.21979440e-03, 9.90780206e-01],
       [9.83839355e-01, 1.61606449e-02],
       [2.83304790e-03, 9.97166952e-01],
       [1.06895121e-01, 8.93104879e-01],
       [6.41086893e-01, 3.58913107e-01],
       [1.63905669e-02, 9.83609433e-01],
       [9.52456926e-01, 4.75430742e-02],
       [6.88063781e-01, 3.11936219e-01],
       [9.98353441e-01, 1.64655883e-03],
       [2.82023149e-01, 7.17976851e-01],
       [1.402176